In [ ]:
# SauceDemo Automation with Playwright
!pip install pytest-playwright allure-pytest pytest-html -q
!playwright install chromium
!playwright install-deps

# Install Allure command line
!wget -q https://github.com/allure-framework/allure2/releases/download/2.24.1/allure-2.24.1.tgz
!tar -xzf allure-2.24.1.tgz
!rm allure-2.24.1.tgz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 MB 19.6 MB/s eta 0:00:00
(node:623) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
164.7 MiB [] 0% 0.0s164.7 MiB [] 0% 110.8s164.7 MiB [] 0% 376.3s164.7 MiB [] 0% 423.9s164.7 MiB [] 0% 340.3s164.7 MiB [] 0% 293.8s164.7 MiB [] 0% 264.3s164.7 MiB [] 0% 244.6s164.7 MiB [] 0% 228.8s164.7 MiB [] 0% 214.2s164.7 MiB [] 0% 201.1s164.7 MiB [] 0% 189.9s164.7 MiB [] 0% 166.6s164.7 MiB [] 0% 150.1s164.7 MiB [] 0% 137.6s164.7 MiB [] 0% 127.5s164.7 MiB [] 0% 119.1s164.7 MiB [] 0% 110.0s164.7 MiB [] 0% 100.5s164.7 MiB [] 0% 91.3s164.7 MiB [] 0% 82.3s164.7 MiB [] 0% 74.8s164.7 MiB [] 0% 68.1s164.7 MiB [] 0% 62.2s164.7 MiB [] 1% 56.8s164.7 MiB [] 1% 51.7s164.7 MiB [] 1% 46.8s164.7 MiB [] 1% 42.2s164.7 MiB [] 1% 3

In [ ]:
import os
os.environ['PATH'] += ':/content/allure-2.24.1/bin'

In [ ]:
# Cell 2: Create Project Structure
import os

# Create directories
os.makedirs('saucedemo_tests', exist_ok=True)
os.makedirs('allure-results', exist_ok=True)

In [ ]:
# Cell 3: Create Test File
test_content = '''import pytest
import allure
from playwright.sync_api import Page, expect
import time


class TestSauceDemo:
    """Test suite for SauceDemo website"""

    BASE_URL = "https://www.saucedemo.com/"

    @pytest.fixture(autouse=True)
    def setup(self, page: Page):
        """Setup: Navigate to SauceDemo before each test"""
        page.goto(self.BASE_URL)
        page.set_viewport_size({"width": 1920, "height": 1080})
        self.page = page
        yield
        # Teardown happens automatically

    def login(self, username: str, password: str):
        """Helper method to perform login"""
        self.page.fill("#user-name", username)
        self.page.fill("#password", password)
        self.page.click("#login-button")

    def reset_app_state(self):
        """Helper method to reset application state"""
        self.page.click("#react-burger-menu-btn")
        self.page.wait_for_selector("#reset_sidebar_link", state="visible")
        self.page.click("#reset_sidebar_link")
        time.sleep(1)
        self.page.click("#react-burger-cross-btn")
        time.sleep(0.5)

    def logout(self):
        """Helper method to logout"""
        self.page.click("#react-burger-menu-btn")
        self.page.wait_for_selector("#logout_sidebar_link", state="visible")
        self.page.click("#logout_sidebar_link")

    @allure.title("Q1: Verify locked out user error message")
    @allure.description("Test login with locked_out_user and verify the error message")
    @allure.severity(allure.severity_level.CRITICAL)
    @pytest.mark.q1
    def test_locked_out_user(self):
        """Q1: Try login with locked_out_user and verify the error message"""

        with allure.step("Attempt login with locked_out_user"):
            self.login("locked_out_user", "secret_sauce")

        with allure.step("Verify error message is displayed"):
            error_element = self.page.locator("[data-test='error']")
            expect(error_element).to_be_visible()

            actual_message = error_element.text_content()
            expected_message = "Epic sadface: Sorry, this user has been locked out."

            allure.attach(
                actual_message,
                name="Error Message",
                attachment_type=allure.attachment_type.TEXT
            )

            assert expected_message == actual_message, \\
                f"Expected: '{expected_message}', but got: '{actual_message}'"

        with allure.step("Capture screenshot of error"):
            screenshot = self.page.screenshot()
            allure.attach(
                screenshot,
                name="locked_out_user_error",
                attachment_type=allure.attachment_type.PNG
            )

        print(f"Q1 Test Passed: Error message verified - '{actual_message}'")

    @allure.title("Q2: Standard user complete purchase journey")
    @allure.description("Login with standard_user, add items, verify checkout and complete purchase")
    @allure.severity(allure.severity_level.CRITICAL)
    @pytest.mark.q2
    def test_standard_user_purchase(self):
        """Q2: Login with standard_user and complete purchase journey"""

        with allure.step("Login with standard_user"):
            self.login("standard_user", "secret_sauce")
            expect(self.page.locator(".inventory_list")).to_be_visible()

        with allure.step("Reset App State"):
            self.reset_app_state()

        product_names = []
        product_prices = []

        with allure.step("Add three items to cart"):
            products = self.page.locator(".inventory_item").all()

            for i in range(3):
                product = products[i]
                name = product.locator(".inventory_item_name").text_content()
                price_text = product.locator(".inventory_item_price").text_content()
                price = float(price_text.replace("$", ""))

                product_names.append(name)
                product_prices.append(price)

                # Click add to cart button
                product.locator("button[id^='add-to-cart']").click()

            allure.attach(
                f"Products: {', '.join(product_names)}\\nPrices: {product_prices}",
                name="Added Products",
                attachment_type=allure.attachment_type.TEXT
            )
            print(f"Added 3 products: {', '.join(product_names)}")

        with allure.step("Navigate to cart"):
            self.page.click(".shopping_cart_link")
            expect(self.page.locator(".cart_list")).to_be_visible()

        with allure.step("Proceed to checkout"):
            self.page.click("#checkout")

        with allure.step("Fill checkout information"):
            self.page.fill("#first-name", "John")
            self.page.fill("#last-name", "Doe")
            self.page.fill("#postal-code", "12345")
            self.page.click("#continue")

        with allure.step("Verify product names on final checkout page"):
            checkout_items = self.page.locator(".inventory_item_name").all()
            checkout_product_names = [item.text_content() for item in checkout_items]

            assert product_names == checkout_product_names, \\
                f"Product names mismatch. Expected: {product_names}, Got: {checkout_product_names}"

            print(f"Product names verified on checkout page")

        with allure.step("Verify total price"):
            item_total_text = self.page.locator(".summary_subtotal_label").text_content()
            item_total_value = float(item_total_text.replace("Item total: $", ""))

            expected_total = sum(product_prices)

            allure.attach(
                f"Expected Total: ${expected_total:.2f}\\nActual Total: ${item_total_value:.2f}",
                name="Price Verification",
                attachment_type=allure.attachment_type.TEXT
            )

            assert abs(expected_total - item_total_value) < 0.01, \\
                f"Total price mismatch. Expected: ${expected_total}, Got: ${item_total_value}"

            print(f"Total price verified: ${item_total_value}")

        with allure.step("Finish purchase"):
            self.page.click("#finish")

        with allure.step("Verify successful order message"):
            success_header = self.page.locator(".complete-header")
            expect(success_header).to_be_visible()

            actual_message = success_header.text_content()
            expected_message = "Thank you for your order!"

            screenshot = self.page.screenshot()
            allure.attach(
                screenshot,
                name="order_success",
                attachment_type=allure.attachment_type.PNG
            )

            assert expected_message == actual_message, \\
                f"Success message mismatch. Expected: '{expected_message}', Got: '{actual_message}'"

            print(f"Order successful: '{actual_message}'")

        with allure.step("Navigate back to products"):
            self.page.click("#back-to-products")

        with allure.step("Reset App State again"):
            self.reset_app_state()

        with allure.step("Logout"):
            self.logout()
            expect(self.page.locator("#login-button")).to_be_visible()

        print("Q2 Test Passed: Complete purchase journey successful")

    @allure.title("Q3: Performance glitch user purchase with filtering")
    @allure.description("Login with performance_glitch_user, filter products, and complete purchase")
    @allure.severity(allure.severity_level.CRITICAL)
    @pytest.mark.q3
    def test_performance_glitch_user_purchase(self):
        """Q3: Login with performance_glitch_user and complete purchase with filtering"""

        with allure.step("Login with performance_glitch_user"):
            self.login("performance_glitch_user", "secret_sauce")
            expect(self.page.locator(".inventory_list")).to_be_visible(timeout=10000)
            time.sleep(2)  # Wait for performance_glitch_user delay

        with allure.step("Reset App State"):
            self.reset_app_state()

        with allure.step("Filter by name (Z to A)"):
            self.page.select_option(".product_sort_container", "za")
            time.sleep(1)

        product_name = ""
        product_price = 0.0

        with allure.step("Select first product and add to cart"):
            first_product = self.page.locator(".inventory_item").first
            product_name = first_product.locator(".inventory_item_name").text_content()
            price_text = first_product.locator(".inventory_item_price").text_content()
            product_price = float(price_text.replace("$", ""))

            first_product.locator("button[id^='add-to-cart']").click()

            allure.attach(
                f"Product: {product_name}\\nPrice: ${product_price}",
                name="Selected Product",
                attachment_type=allure.attachment_type.TEXT
            )

            print(f"Added product: {product_name} - ${product_price}")

        with allure.step("Navigate to cart"):
            self.page.click(".shopping_cart_link")
            expect(self.page.locator(".cart_list")).to_be_visible()

        with allure.step("Proceed to checkout"):
            self.page.click("#checkout")

        with allure.step("Fill checkout information"):
            self.page.fill("#first-name", "Jane")
            self.page.fill("#last-name", "Smith")
            self.page.fill("#postal-code", "54321")
            self.page.click("#continue")

        with allure.step("Verify product name on final checkout page"):
            checkout_item = self.page.locator(".inventory_item_name").first
            checkout_product_name = checkout_item.text_content()

            assert product_name == checkout_product_name, \\
                f"Product name mismatch. Expected: {product_name}, Got: {checkout_product_name}"

            print(f"Product name verified: {checkout_product_name}")

        with allure.step("Verify total price"):
            item_total_text = self.page.locator(".summary_subtotal_label").text_content()
            item_total_value = float(item_total_text.replace("Item total: $", ""))

            allure.attach(
                f"Expected Total: ${product_price:.2f}\\nActual Total: ${item_total_value:.2f}",
                name="Price Verification",
                attachment_type=allure.attachment_type.TEXT
            )

            assert abs(product_price - item_total_value) < 0.01, \\
                f"Total price mismatch. Expected: ${product_price}, Got: ${item_total_value}"

            print(f"Total price verified: ${item_total_value}")

        with allure.step("Finish purchase"):
            self.page.click("#finish")

        with allure.step("Verify successful order message"):
            success_header = self.page.locator(".complete-header")
            expect(success_header).to_be_visible()

            actual_message = success_header.text_content()
            expected_message = "Thank you for your order!"

            screenshot = self.page.screenshot()
            allure.attach(
                screenshot,
                name="order_success_performance_user",
                attachment_type=allure.attachment_type.PNG
            )

            assert expected_message == actual_message, \\
                f"Success message mismatch. Expected: '{expected_message}', Got: '{actual_message}'"

            print(f"Order successful: '{actual_message}'")

        with allure.step("Navigate back to products"):
            self.page.click("#back-to-products")

        with allure.step("Reset App State again"):
            self.reset_app_state()

        with allure.step("Logout"):
            self.logout()
            expect(self.page.locator("#login-button")).to_be_visible()

        print("Q3 Test Passed: Performance glitch user purchase successful")
'''

with open('saucedemo_tests/test_saucedemo.py', 'w') as f:
    f.write(test_content)

print("Test file created: saucedemo_tests/test_saucedemo.py")


Test file created: saucedemo_tests/test_saucedemo.py


In [ ]:
# Cell 4: Create pytest.ini
pytest_ini_content = '''[pytest]
markers =
    q1: marks tests for Question 1 (locked_out_user)
    q2: marks tests for Question 2 (standard_user purchase)
    q3: marks tests for Question 3 (performance_glitch_user purchase)

addopts =
    --alluredir=allure-results
    --headed
    -v
    -s
'''

with open('pytest.ini', 'w') as f:
    f.write(pytest_ini_content)

print("pytest.ini created")

# Cell 5: Create conftest.py for Playwright
conftest_content = '''import pytest
from playwright.sync_api import sync_playwright


@pytest.fixture(scope="function")
def page(browser):
    """Create a new page for each test"""
    context = browser.new_context(
        viewport={'width': 1920, 'height': 1080}
    )
    page = context.new_page()
    yield page
    context.close()


@pytest.fixture(scope="session")
def browser():
    """Create browser instance for the session"""
    with sync_playwright() as p:
        browser = p.chromium.launch(
            headless=True,
            args=['--no-sandbox', '--disable-dev-shm-usage']
        )
        yield browser
        browser.close()
'''

with open('saucedemo_tests/conftest.py', 'w') as f:
    f.write(conftest_content)

print("conftest.py created")

pytest.ini created
conftest.py created


In [ ]:
# Cell 6: Create requirements.txt
requirements_content = '''pytest==7.4.3
pytest-playwright==0.4.3
playwright==1.40.0
allure-pytest==2.13.2
pytest-html==4.1.1
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements_content)

print("requirements.txt created")

requirements.txt created


In [ ]:
# Cell 7: Run All Tests
print("\\n" + "="*60)
print("RUNNING ALL TESTS SEQUENTIALLY")
print("="*60 + "\\n")

!pytest saucedemo_tests/test_saucedemo.py -v -s

\n============================================================
RUNNING ALL TESTS SEQUENTIALLY
============================================================\n
============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
metadata: {'Python': '3.12.12', 'Platform': 'Linux-6.6.105+-x86_64-with-glibc2.35', 'Packages': {'pytest': '8.4.2', 'pluggy': '1.6.0'}, 'Plugins': {'playwright': '0.7.2', 'allure-pytest': '2.15.3', 'base-url': '2.1.0', 'html': '4.1.1', 'metadata': '3.1.1', 'typeguard': '4.4.4', 'langsmith': '0.4.59', 'anyio': '4.12.0'}, 'Base URL': ''}
rootdir: /content
configfile: pytest.ini
plugins: playwright-0.7.2, allure-pytest-2.15.3, base-url-2.1.0, html-4.1.1, metadata-3.1.1, typeguard-4.4.4, langsmith-0.4.59, anyio-4.12.0
collected 3 items                                                              

saucedemo_tests/test_saucedemo.py::TestSauceDemo::test_l

In [ ]:
# Cell 8: Run Q1 Only
print("\\n" + "="*60)
print("RUNNING Q1 ONLY")
print("="*60 + "\\n")

!pytest saucedemo_tests/test_saucedemo.py::TestSauceDemo::test_locked_out_user -v -s

\n============================================================
RUNNING Q1 ONLY
============================================================\n
============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
metadata: {'Python': '3.12.12', 'Platform': 'Linux-6.6.105+-x86_64-with-glibc2.35', 'Packages': {'pytest': '8.4.2', 'pluggy': '1.6.0'}, 'Plugins': {'playwright': '0.7.2', 'allure-pytest': '2.15.3', 'base-url': '2.1.0', 'html': '4.1.1', 'metadata': '3.1.1', 'typeguard': '4.4.4', 'langsmith': '0.4.59', 'anyio': '4.12.0'}, 'Base URL': ''}
rootdir: /content
configfile: pytest.ini
plugins: playwright-0.7.2, allure-pytest-2.15.3, base-url-2.1.0, html-4.1.1, metadata-3.1.1, typeguard-4.4.4, langsmith-0.4.59, anyio-4.12.0
collected 1 item                                                               

saucedemo_tests/test_saucedemo.py::TestSauceDemo::test_locked_out_user 

In [ ]:
# Cell 9: Run Q2 Only
print("\\n" + "="*60)
print("RUNNING Q2 ONLY")
print("="*60 + "\\n")

!pytest saucedemo_tests/test_saucedemo.py::TestSauceDemo::test_standard_user_purchase -v -s

\n============================================================
RUNNING Q2 ONLY
============================================================\n
============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
metadata: {'Python': '3.12.12', 'Platform': 'Linux-6.6.105+-x86_64-with-glibc2.35', 'Packages': {'pytest': '8.4.2', 'pluggy': '1.6.0'}, 'Plugins': {'playwright': '0.7.2', 'allure-pytest': '2.15.3', 'base-url': '2.1.0', 'html': '4.1.1', 'metadata': '3.1.1', 'typeguard': '4.4.4', 'langsmith': '0.4.59', 'anyio': '4.12.0'}, 'Base URL': ''}
rootdir: /content
configfile: pytest.ini
plugins: playwright-0.7.2, allure-pytest-2.15.3, base-url-2.1.0, html-4.1.1, metadata-3.1.1, typeguard-4.4.4, langsmith-0.4.59, anyio-4.12.0
collected 1 item                                                               

saucedemo_tests/test_saucedemo.py::TestSauceDemo::test_standard_user_pu

In [ ]:
# Cell 10: Run Q3 Only
print("\\n" + "="*60)
print("RUNNING Q3 ONLY")
print("="*60 + "\\n")

!pytest saucedemo_tests/test_saucedemo.py::TestSauceDemo::test_performance_glitch_user_purchase -v -s

\n============================================================
RUNNING Q3 ONLY
============================================================\n
============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
metadata: {'Python': '3.12.12', 'Platform': 'Linux-6.6.105+-x86_64-with-glibc2.35', 'Packages': {'pytest': '8.4.2', 'pluggy': '1.6.0'}, 'Plugins': {'playwright': '0.7.2', 'allure-pytest': '2.15.3', 'base-url': '2.1.0', 'html': '4.1.1', 'metadata': '3.1.1', 'typeguard': '4.4.4', 'langsmith': '0.4.59', 'anyio': '4.12.0'}, 'Base URL': ''}
rootdir: /content
configfile: pytest.ini
plugins: playwright-0.7.2, allure-pytest-2.15.3, base-url-2.1.0, html-4.1.1, metadata-3.1.1, typeguard-4.4.4, langsmith-0.4.59, anyio-4.12.0
collected 1 item                                                               

saucedemo_tests/test_saucedemo.py::TestSauceDemo::test_performance_glit

In [ ]:
# Cell 11: Generate and Display Allure Report
print("\\n" + "="*60)
print("GENERATING ALLURE REPORT")
print("="*60 + "\\n")

!allure generate allure-results --clean -o allure-report

# Install a simple HTTP server to view the report
!pip install -q aiohttp

# Create a simple script to serve the report
serve_script = '''
import os
from aiohttp import web
import asyncio

async def serve_report(request):
    file_path = os.path.join('allure-report', 'index.html')
    with open(file_path, 'r') as f:
        return web.Response(text=f.read(), content_type='text/html')

async def serve_static(request):
    path = request.match_info['path']
    file_path = os.path.join('allure-report', path)

    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            content_type = 'text/html'
            if path.endswith('.js'):
                content_type = 'application/javascript'
            elif path.endswith('.css'):
                content_type = 'text/css'
            elif path.endswith('.json'):
                content_type = 'application/json'

            return web.Response(body=f.read(), content_type=content_type)
    return web.Response(status=404)

app = web.Application()
app.router.add_get('/', serve_report)
app.router.add_get('/{path:.*}', serve_static)

web.run_app(app, host='0.0.0.0', port=8080)
'''

with open('serve_report.py', 'w') as f:
    f.write(serve_script)

print("Allure report generated!")

\n============================================================
GENERATING ALLURE REPORT
============================================================\n
Report successfully generated to allure-report
Allure report generated!


In [ ]:
# Cell 12: Create ZIP of Reports
print("\\n" + "="*60)
print("CREATING ZIP FILE FOR DOWNLOAD")
print("="*60 + "\\n")

!zip -r saucedemo_reports.zip allure-report allure-results

from google.colab import files
print("\\nDownloading reports ZIP file...")
files.download('saucedemo_reports.zip')

\n============================================================
CREATING ZIP FILE FOR DOWNLOAD
============================================================\n
  adding: allure-report/ (stored 0%)
  adding: allure-report/history/ (stored 0%)
  adding: allure-report/history/retry-trend.json (stored 0%)
  adding: allure-report/history/history.json (deflated 62%)
  adding: allure-report/history/history-trend.json (deflated 18%)
  adding: allure-report/history/categories-trend.json (stored 0%)
  adding: allure-report/history/duration-trend.json (stored 0%)
  adding: allure-report/styles.css (deflated 50%)
  adding: allure-report/index.html (deflated 54%)
  adding: allure-report/widgets/ (stored 0%)
  adding: allure-report/widgets/retry-trend.json (stored 0%)
  adding: allure-report/widgets/history-trend.json (deflated 18%)
  adding: allure-report/widgets/categories.json (stored 0%)
  adding: allure-report/widgets/environment.json (stored 0%)
  adding: allure-report/widgets/suites.json (deflat

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Cell 13: Display Test Summary
print("\\n" + "="*60)
print("TEST EXECUTION SUMMARY")
print("="*60)

!pytest saucedemo_tests/test_saucedemo.py --collect-only

print("\\nAll tests completed!")
print("\\nTest Results:")
print("- Q1: Locked out user error message verification")
print("- Q2: Standard user complete purchase journey")
print("- Q3: Performance glitch user with filtering")

\n============================================================
TEST EXECUTION SUMMARY
============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
metadata: {'Python': '3.12.12', 'Platform': 'Linux-6.6.105+-x86_64-with-glibc2.35', 'Packages': {'pytest': '8.4.2', 'pluggy': '1.6.0'}, 'Plugins': {'playwright': '0.7.2', 'allure-pytest': '2.15.3', 'base-url': '2.1.0', 'html': '4.1.1', 'metadata': '3.1.1', 'typeguard': '4.4.4', 'langsmith': '0.4.59', 'anyio': '4.12.0'}, 'Base URL': ''}
rootdir: /content
configfile: pytest.ini
plugins: playwright-0.7.2, allure-pytest-2.15.3, base-url-2.1.0, html-4.1.1, metadata-3.1.1, typeguard-4.4.4, langsmith-0.4.59, anyio-4.12.0
collected 3 items                                                              

<Dir content>
  <Dir saucedemo_tests>
    <Module test_saucedemo.py>
      <Class TestSauceDemo>
        Test suite for SauceDe